In [1]:
# !pip install tensorflow
# !pip install matplotlib
# !pip install scikit-learn
# !pip install numpy
# !pip install pandas

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, Callback
from sklearn.model_selection import KFold
from tensorflow.keras.applications import ResNet50, DenseNet121, VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator


2024-05-04 00:51:16.700126: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 00:51:16.700231: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 00:51:16.826089: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def load_data(csv_path):
    return pd.read_csv(csv_path)
    #dataframe

In [4]:
def add_png_suffix(image_id):
    return f"{image_id}.png"

In [5]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 25:
        lr *= 0.1
    elif epoch >15:
        lr *= 0.5
    return  lr

In [6]:
def create_model(base_model):
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(2)  # Assuming two outputs: angle and speed
    ])
    model.compile(optimizer = Adam(learning_rate = lr_schedule(0)), loss = 'mse', metrics=['accuracy'])
    return model

In [7]:
def plot_training_history(history, model):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.title(f'{model.name} Training and Validation Accuracy')
    plt.legend(loc='lower right')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.title(f'{model.name} Training and Validation Loss')
    plt.legend(loc='upper right')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.savefig(f'{model.name}_loss_accuracy_curves.png')  # Save the loss plot as an image file
    plt.show()

In [8]:
datagen = ImageDataGenerator(
        rescale = 1./255,  # Normalize the image
        width_shift_range = 0.2,  # Horizontal offset range
        height_shift_range = 0.2,  # Range of vertical offset
        shear_range = 0.2,  # Shear strength
        zoom_range = 0.2,  #Random scaling range
        brightness_range=[0.6, 1.4],  # Randomly adjust the range of image brightness
        fill_mode = 'nearest',  # Method to fill newly created pixels
        validation_split=0.2
    )

In [9]:
def create_train_generator(dataframe, image_directory, target_size=(128, 128), batch_size=32):
    train_generator = datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = image_directory,
        x_col = 'image_id',
        y_col = ['angle', 'speed'],
        class_mode = 'raw',  # 因为这是一个回归问题，不是分类问题
        target_size = target_size,  # 假设我们希望所有图像都调整为320x240
        batch_size = batch_size,
#         subset='training'
    )

    return train_generator

In [10]:
def create_validation_generator(dataframe, image_directory, target_size=(128, 128), batch_size=32):
    validation_generator = datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = image_directory,
        x_col = 'image_id',
        y_col = ['angle', 'speed'],
        class_mode = 'raw',
        target_size = target_size,
        batch_size = batch_size,
#         subset = 'validation' #Specify that this is verification data
    )

    return validation_generator

In [11]:
class TrainingMonitor(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f"End of epoch {epoch + 1}")
        print(f"Training loss: {logs.get('loss')}, Training accuracy: {logs.get('accuracy')}")
        print(f"Validation loss: {logs.get('val_loss')}, Validation accuracy: {logs.get('val_accuracy')}")

In [12]:
def setup_callbacks():
    lr_scheduler = LearningRateScheduler(lr_schedule)
    early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1, mode='min', restore_best_weights=True)
    monitor = TrainingMonitor()
    return [lr_scheduler, early_stopper, monitor]


In [13]:
def train_evaluate_model(df_train, df_val, image_file, base_model, target_size=(128,128), batch_size=32, epochs=30):
    model = create_model(base_model)
    lr_scheduler = LearningRateScheduler(lr_schedule)
    callbacks = setup_callbacks()
    steps_per_epoch = (len(df_train) // batch_size)
    validation_steps = int(np.ceil(len(df_val) / batch_size))

    train_generator = create_train_generator(df_train, image_file, target_size=target_size, batch_size=batch_size)
    val_generator = create_validation_generator(df_val, image_file, target_size=target_size, batch_size=batch_size)

    history = model.fit(
        train_generator,
        steps_per_epoch = steps_per_epoch,
        validation_data = val_generator,
        validation_steps = validation_steps,
        epochs = epochs,
        callbacks = callbacks
    )

    loss, accuracy = model.evaluate(val_generator)
    print(f"Evaluation results -- Loss: {loss}, Accuracy: {accuracy}")

    return history, loss, accuracy, model

In [14]:
def perform_kfold_cross_validation(full_df, image_file, base_model, k=5, target_size=(128,128), batch_size=32, epochs=30):
    # Initialize k-fold cross-validation
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)
    fold_results = []
    best_accuracy = 0
    best_model = None
    best_fold = None

    print("Starting training process...")

    for fold, (train_idx, val_idx) in enumerate(kfold.split(full_df), start=1):
        print(f"Fold {fold}/{k}:")
        print(f'Number of training samples: {len(train_idx)}, Number of validation samples: {len(val_idx)}')

        # Split the data into training and validation
        df_train = full_df.iloc[train_idx]
        df_val = full_df.iloc[val_idx]

        # Train and evaluate the model for the current fold
        history, loss, accuracy, model = train_evaluate_model(
            df_train, df_val, image_file, base_model,
            target_size=target_size, batch_size=batch_size, epochs=epochs
        )

        # Update the best model if the current fold's accuracy is higher
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model
            best_fold = fold
            print(f'Updated best model at fold {fold} with accuracy: {accuracy}')

        fold_results.append({
            'fold': fold,
            'history': history,
            'loss': loss,
            'accuracy': accuracy
        })

    # Save the best model
    if best_model is not None:
        model_path = os.path.join(os.getcwd(), f'{best_model.input_names[0]}_model.h5')
        # Check if the file exists and remove it if it does
        if os.path.exists(model_path):
            os.remove(model_path)
        best_model.save(model_path)
        print(f"Training completed. Best model saved from fold {best_fold} with validation accuracy: {best_accuracy:.2f} at {model_path}")
    else:
        print("No model improvement observed across folds.")

    return fold_results



In [15]:
def train_multiple_models(full_df, image_file, base_models, k=5, target_size=(128,128), batch_size=32, epochs=30):
    # Returns: A dictionary with models' names as keys and their performance metrics as values.
    results = {}

    for model_creator in base_models:
        model = model_creator()
        print(f"Training model: {model.name}")

        result = perform_kfold_cross_validation(full_df, image_file, model, k = k, target_size = target_size, batch_size = batch_size, epochs = epochs)

        results[model.name] = result

    return results

In [16]:
def main():
    csv_file = '/kaggle/input/machine-learning-in-science-ii-2024/training_norm.csv'
    image_file = '/kaggle/input/machine-learning-in-science-ii-2024/training_data/training_data'
    k = 2
    target_size = (128,128)
    batch_size = 32
    epochs = 1

    full_df = load_data(csv_file)

    full_df['image_id'] = full_df['image_id'].apply(add_png_suffix)

    base_models = [
      lambda: ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3)),
      lambda: DenseNet121(weights='imagenet', include_top=False, input_shape=(128, 128, 3)),
      lambda: VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    ]

    results = train_multiple_models(full_df, image_file, base_models, k=k, target_size=target_size, batch_size=batch_size, epochs=epochs)

    print(results)

In [17]:
main()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training model: resnet50
Starting training process...
Fold 1/2:
Number of training samples: 6896, Number of validation samples: 6897
Found 6896 validated image filenames.
Found 6897 validated image filenames.


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1714783984.392472     109 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1714783984.483826     109 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


  6/215 ━━━━━━━━━━━━━━━━━━━━ 20:42 6s/step - accuracy: 0.5816 - loss: 1.4775  

W0000 00:00:1714784014.261400     109 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 529ms/step - accuracy: 0.8479 - loss: 0.2013

...

W0000 00:00:1714784361.411621     109 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step - accuracy: 0.8708 - loss: 0.1515

W0000 00:00:1714784389.615916     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1714784444.565838     108 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


End of epoch 1
Training loss: 0.05857301503419876, Training accuracy: 0.9190094470977783
Validation loss: 0.15398447215557098, Validation accuracy: 0.7558004856109619
215/215 ━━━━━━━━━━━━━━━━━━━━ 190s 645ms/step - accuracy: 0.8710 - loss: 0.1510 - val_accuracy: 0.7558 - val_loss: 0.1540 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 1.
216/216 ━━━━━━━━━━━━━━━━━━━━ 54s 249ms/step - accuracy: 0.7527 - loss: 0.1544
Evaluation results -- Loss: 0.15400806069374084, Accuracy: 0.7558004856109619
Updated best model at fold 2 with accuracy: 0.7558004856109619


AttributeError: 'Sequential' object has no attribute 'input_names'